In [2]:
from sasha_tools import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression

In [13]:
df_text = pd.read_csv('text_categories.csv')
df_catboost = pd.read_csv('cb_score.csv')
df_features = pd.read_csv('linear_features.csv')

df = df_text.merge(df_catboost, on = 'APPID', how = 'outer').merge(df_features, on = 'APPID', how = 'outer')

In [14]:
df['date'] = pd.to_datetime(df['APPDATE'].apply(lambda x : str(x)[:10]), format = '%d.%m.%Y')

In [19]:
model = LogisticRegression(penalty='none')
train_indx = (df['date'] <= '2022-03-31')

In [17]:
df.columns

Index(['APPID', 'iphone_category', 'xiaomi_category', 'other_category',
       'cb_score_map', 'APPDATE', 'EMAIL_ftr', 'APP_rank_ftr', 'LIM_ftr',
       'LIM_zero_flag', 'daytime_flag', 'TARGET', 'date'],
      dtype='object')

In [20]:
model_columns = ['iphone_category', 'xiaomi_category', 'other_category',
                 'cb_score_map', 'EMAIL_ftr', 'APP_rank_ftr', 'LIM_ftr',
                 'LIM_zero_flag', 'daytime_flag']

model.fit(df.loc[train_indx, model_columns], df.loc[train_indx, 'TARGET'])

LogisticRegression(penalty='none')

In [25]:
df['predict'] = model.predict_proba(df[model_columns])[:, 1]

In [26]:
df[~train_indx].to_csv('eval_result.csv', index = False)